# Futures

При помощи Future обычно описывается объект, к которому можно обратиться за результатом,но вычисление которого может быть не завершено на данный момент.

В питоне Future инкапсулирует выполнение некоторого callable обьекта. Имеет следующий интерфейс: 
```python
class concurrent.futures.Future:
  def done()  
  def running()  
  def cancelled()  
```  
Фактически позволяют проверить состояние Future, кроме трёх состояний выше, Future ещё может находится в очереди на выполнение.
```python
 def cancel()
```
Позволяет отменить Future, выполнение которого ещё не началось
```python
 def result(timeout=None)
 def exception(timeout=None)
```
Два метода позволяющие получить результат Future, если вы зовёте result,  а внутри было брошено исключение, то оно будет брошено повторно. Если Future было отменено, будет брошено CancelledError, если время ожидания достигнуто, будет брошено concurrent.futures.TimeoutError
```python
  def add_done_callback(fn)
```
 Добавляет callback к Future, callback-и зовуться в той же очерёдности, что и были добавлены. Exception-ы брошеные callback-ом, логируются и игнорируются.Если Future завершено, то callback будет вызван немедленно.

In [9]:
from concurrent.futures import ThreadPoolExecutor
from time import sleep
 
def return_after_5_secs(message):
    sleep(5)
    return message
 
pool = ThreadPoolExecutor(3)
 
future = pool.submit(return_after_5_secs, ("hello"))
print(type(future))
print(future.done())
sleep(5)
print(future.done())
print(future.result())
print(future.done())

<class 'concurrent.futures._base.Future'>
False
True
hello
True


#### Executors
Как видно из примера выше, чтобы исполнить Future нам необходим некоторый "исполнитель" в стандартной поставке таких исполнителей два ThreadPoolExecutor и ProcessPoolExecutor. Интерфейс: 
```python
     def submit(fn, *args, **kwargs)
```   
 Добавляет в очередь исполнения fn(\*args, \**kwargs), возвращает future.
```python
     def map(func, *iterables, timeout=None, chunksize=1)
```   
  Ждёт результаты
```python
     def shutdown(wait=True)
```

In [10]:
from concurrent.futures import ProcessPoolExecutor
from time import sleep
 
with ProcessPoolExecutor(3) as pool: 
    future = pool.submit(return_after_5_secs, ("hello"))
    print(future.done())
    sleep(5)
    print(future.done())
    print("Result: " + future.result())
    pool.shutdown()
    pool.submit(return_after_5_secs, ("hello"))


False
False
Result: hello


RuntimeError: cannot schedule new futures after shutdown

#### Полезные методы модуля
```python
concurrent.futures.wait(fs, timeout=None, return_when=ALL_COMPLETED)
```
 Разделяет fs на два списка - done/not_done, return_when может принимать значения: FIRST_COMPLETED/FIRST_EXCEPTION/ALL_COMPLETED

```python
concurrent.futures.as_completed(fs, timeout=None)
```
 позволяет просто проитерироваться по законченым Future


# Многозадачность

**Вытесняющая**  
Вид многозадачности, в котором операционная система сама передает управление от одной выполняемой программы другой в случае завершения операций ввода-вывода, возникновения событий в аппаратуре компьютера, истечения таймеров и квантов времени, или же поступлений тех или иных сигналов от одной программы к другой. В этом виде многозадачности процессор может быть переключен с исполнения одной программы на исполнение другой без всякого пожелания первой программы и буквально между любыми двумя инструкциями в её коде. Распределение процессорного времени осуществляется планировщиком процессов. К тому же каждой задаче может быть назначен пользователем или самой операционной системой определенный приоритет, что обеспечивает гибкое управление распределением процессорного времени между задачами (например, можно снизить приоритет ресурсоёмкой программе, снизив тем самым скорость её работы, но повысив производительность фоновых процессов)

**Кооперативная многозадачность**  
Тип многозадачности, при котором следующая задача выполняется только после того, как текущая задача явно объявит себя готовой отдать процессорное время другим задачам. 

In [9]:
from time import sleep
def gen(): 
    for i in range(3): 
        sleep(0.5)
        yield i
g = gen()    
v_1 = next(g)
# some work
v_2 = next(g)
#list(gen())

[0, 1, 2]

In [2]:
import asyncio
import datetime

async def display_date(loop):
    end_time = loop.time() + 5.0
    while True:
        print(datetime.datetime.now())
        if (loop.time() + 1.0) >= end_time:
            break
        await asyncio.sleep(1)

loop = asyncio.get_event_loop()
# Blocking call which returns when the display_date() coroutine is done
loop.run_until_complete(display_date(loop))
loop.close()

2017-11-08 19:37:56.308961
2017-11-08 19:37:57.310241
2017-11-08 19:37:58.311930
2017-11-08 19:37:59.314594
2017-11-08 19:38:00.316820


In [ ]:
import asyncio
import asyncio_redis

@asyncio.coroutine
def my_subscriber(channels):
    # Create connection
    connection = yield from asyncio_redis.Connection.create(host='localhost', port=6379)
    # Create subscriber.
    subscriber = yield from connection.start_subscribe()
    # Subscribe to channel.
    yield from subscriber.subscribe(channels)
    # Wait for incoming events.
    while True:
        reply = yield from subscriber.next_published()
        print('Received: ', repr(reply.value), 'on channel', reply.channel)

loop = asyncio.get_event_loop()
asyncio.async(my_subscriber('channel-1'))
asyncio.async(my_subscriber('channel-2'))
loop.run_forever()

Как же работает магия.

In [2]:
def replacement_socket_recv(the_socket):
    loop = asyncio.get_event_loop()
    loop.register_my_interest_in(the_socket,
                    for_the_operation='recv')
    yield  # wait for the event loop to resume me
    return the_socket.recv()  # (2)
# then monkey-patch this into the right place

Что может корутина:  

* **result = await future or result = yield from future** -
  приостанавливает корутину, пока Future не будет выполнено.

* **result = await coroutine or result = yield from coroutine**- ожидать результата другой корутины
* **return expression **-вернуть результат в ждущую корутину
* **raise exception**- бросить исключение, в ждущей корутине

### Альтернативы
Gevent/eventlet/twisted monkey-patch + implicit

In [ ]:
# Implicit
result = self.make_rpc_call() # implicit switch here
# Explicit
result = yield from self.make_rpc_call()

# Акторы
Это ещё один механизм организации параллелизма, в питоне особо не прижились. Но давайте кратко остановимся. 

В акторной модели, все компоненты являются акторами(как в обьектно-ориентированной всё - обьекты.). Акторы общаются через механизм сообщений. В ответ на получение сообщения актор может: 
* создать конечное число других акторов
* отправить конечное число сообщений другим акторам
* изменить своё состояние так, чтобы по другому реагировать на следующией сообщения

В модели интересно то, что она отделяет посылающую сторону, от стороны получения сообщений, что позволяет реализовать ассинхронную коммуникацию.

In [ ]:
# https://github.com/kquick/Thespian/blob/master/examples/hellogoodbye.py
# does not work in notebook( 
import logging
from logsetup import logcfg
from datetime import timedelta
from thespian.actors import *

class Hello(Actor):
    def receiveMessage(self, message, sender):
        logging.info('Hello got: %s', message)
        if message == 'are you there?':
            world = self.createActor(World)
            worldmsg = (sender, 'Hello,')
            self.send(world, worldmsg)

class World(Actor):
    def receiveMessage(self, message, sender):
        if isinstance(message, tuple):
            orig_sender, pre_world = message
            self.send(orig_sender, pre_world + ' world!')

class Goodbye(Actor):
    def receiveMessage(self, message, sender):
        self.send(sender, 'Goodbye')


def run_example(systembase=None):
    asys = ActorSystem(systembase, logDefs=logcfg)
    hello = ActorSystem().createActor(Hello)
    goodbye = ActorSystem().createActor(Goodbye)
    greeting = ActorSystem().ask(hello, 'are you there?', timedelta(seconds=1.5))
    print(greeting + '\n' + ActorSystem().ask(goodbye, None,
                                              timedelta(milliseconds=100)))
    ActorSystem().shutdown()

if __name__ == "__main__":
    import sys
    run_example(sys.argv[1] if len(sys.argv) > 1 else None)